In [2]:
import json
import httpx
from langdetect import detect
from num2words import num2words
from googletrans import Translator
from tqdm.autonotebook import tqdm
# from ofa.ofa_infer import OFAInference
# from translator import EnViVinAITranslator, JaEnMarianTranslator

In [4]:
ofa = OFAInference()

/home/phucpx/miniconda3/envs/phucpx/lib/python3.7/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/phucpx/miniconda3/envs/phucpx/lib/python3.7/site-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


In [5]:
# ofa_cn = OFAInference(pretrained_path="models/ofa_cn_large.pt", bpe_dir="ofa/utils/BERT_CN_dict")

In [3]:
envi_translator = EnViVinAITranslator()
jaem_translator = JaEnTranslator()

In [4]:
timeout = httpx.Timeout(10) 

gg_translator = Translator(timeout=timeout)

In [4]:
envi_translator.translate_vi2en("có bao nhiêu người đang đứng bán phở?")

'How many people are standing there selling noodles?'

In [5]:
with open('data/test/official_evjvqa_public_test.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
    
test_images = test_data['images']
test_annotations = test_data['annotations']

len(test_images), len(test_annotations)

(558, 5015)

In [5]:
# img_path ='./data/test/public-test-images'

# imgid2imgpath = {}
# lang_count = {}

# for i in range(len(test_images)):
#     timg = test_images[i]
#     imgid2imgpath[timg['id']] = f"{img_path}/{timg['filename']}"
    
# for i in range(len(test_annotations)):
#     anno = test_annotations[i]
#     anno['img_path'] = imgid2imgpath[anno['image_id']]
#     lang = detect(anno['question'])
#     anno['language'] = lang
#     if lang in lang_count:
#         lang_count[lang] += 1
#     else:
#         lang_count[lang] = 1

In [6]:
# lang_count

In [7]:
# with open('./data/test/official_evjvqa_public_test_lang.json', 'w', encoding='utf=8') as f:
#     json.dump(test_data, f, indent=4, ensure_ascii=False)

## RUNNING

In [5]:
with open('data/test/official_evjvqa_public_test_lang.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
    
test_images = test_data['images']
test_annotations = test_data['annotations']

In [10]:
# answer_list = []

In [46]:
answer_dict = {}

start_idx = 4800
end_idx = 5100

for i, anno in tqdm(enumerate(test_annotations)):
    if start_idx <= i and i < end_idx:
        if anno['language'] == 'en':
            answer = ofa.ofa_inference(anno['img_path'], anno['question'])
            split_ans = answer.split()
            ans = []
            for w in split_ans:
                try:
                    ans.append(num2words(w))
                except:
                    ans.append(w)
            answer = ' '.join(ans)
            answer_dict[anno['id']] = f"{anno['question']} {answer}"
        elif anno['language'] == 'vi':
            en_question = gg_translator.translate(anno['question'], src='vi', dest='en').text
            en_answer = ofa.ofa_inference(anno['img_path'], en_question)
            vi_answer = gg_translator.translate(en_answer, src='en', dest='vi').text
            split_ans = vi_answer.split()
            ans = []
            for w in split_ans:
                try:
                    ans.append(num2words(w))
                except:
                    ans.append(w)
            vi_answer = ' '.join(ans)
            answer_dict[anno['id']] = f"{anno['question']} {vi_answer}"
    #         en_question = envi_translator.translate_vi2en(anno['question'])
    #         en_answer = ofa.ofa_inference(anno['img_path'], en_question)
    #         vi_answer = envi_translator.translate_en2vi(en_answer)
    #         answer_dict[anno['id']] = f"{anno['question']} {vi_answer}"

    #         answer_dict[anno['id']] = ''
        elif anno['language'] == 'ja':
            en_question = gg_translator.translate(anno['question'], src='ja', dest='en').text
            en_answer = ofa.ofa_inference(anno['img_path'], en_question)
            ja_answer = gg_translator.translate(en_answer, src='en', dest='ja').text    
            answer_dict[anno['id']] = f"{anno['question']} {ja_answer}"
            

answer_list.append(answer_dict)

0it [00:00, ?it/s]

In [47]:
len(answer_list)

17

In [48]:
return_dict = {}

for i in range(len(answer_list)):
    data = answer_list[i]
    for k, v in data.items():
        if k not in return_dict:
            return_dict[k] = v
            
len(return_dict)

5015

In [50]:
with open('./outputs/results.json', 'w', encoding='utf-8') as f:
    json.dump(return_dict, f, indent=4, ensure_ascii=False)

In [16]:
assert len(test_annotations) == len(answer_dict)

In [25]:
lang_dict = {'vi': 0, 'ja': 0, 'en': 0}
for anno in test_annotations:
    lang_dict[anno['language']] += 1

lang_dict

{'vi': 1678, 'ja': 1651, 'en': 1686}

In [52]:
import visen
from vncorenlp import VnCoreNLP
from unicodedata import normalize as unl

In [53]:
annotator = VnCoreNLP(
    '/home/phucpx/vinbdi/MRC-for-OE-ABSA/models/pretrained/VnCoreNLP/VnCoreNLP-1.1.1.jar', 
    annotators="wseg", 
    max_heap_size='-Xmx2g'
)

def normalize(text):
    text = unl('NFKC', text)
    text = visen.clean_tone(text)
    
    seg_text = annotator.tokenize(text)
    listwords = []
    for sent in seg_text:
        for word in sent:
            listwords.append(word.replace('_', ' '))
    
    text = ' '.join(listwords)
    
    return text

In [1]:
with open('./outputs/results.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
for k, v in data.items():
    data[k] = normalize(v)
    
with open('./outputs/results.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

## Infer Training Set

In [6]:
with open('data/train/evjvqa_train_lang.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
images = data['images']
annotations = data['annotations']

In [12]:
# id2path = {}

# for img in images:
#     id2path[img['id']] = f"data/train/train-images/{img['filename']}"

# for anno in annotations:
#     anno['img_path'] = id2path[anno['image_id']]

# with open('data/train/evjvqa_train_lang.json', 'w', encoding='utf-8') as f:
#     json.dump(data, f, indent=4, ensure_ascii=False)

In [24]:
answer_dict = {}

for i, anno in tqdm(enumerate(annotations)):
    if anno['language'] == 'en':
        answer = ofa.ofa_inference(anno['img_path'], anno['question'])
        split_ans = answer.split()
        ans = []
        for w in split_ans:
            try:
                ans.append(num2words(w))
            except:
                ans.append(w)
        answer = ' '.join(ans)
        answer_dict[anno['id']] = f"{answer}"
        answer_dict[anno['id']] = f"{anno['question']} {answer}"


answer_dict

0it [00:00, ?it/s]

/home/phucpx/vinbdi/EVJVQA/ofa/models/sequence_generator.py:695: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = bbsz_idx // beam_size


{0: 'red',
 1: 'a bunch of green grapes',
 2: 'bowls of fruit',
 3: 'plan',
 4: 'reading',
 5: 'plant',
 6: 'purse',
 7: 'a store',
 8: 'shopping',
 9: 'shopping',
 10: 'a woman in pink',
 11: 'excited',
 12: 'christmas decorations',
 13: 'taking picture',
 14: 'tires',
 15: 'orange',
 16: 'blue and white',
 17: 'broccoli',
 18: 'shopping',
 19: 'white',
 20: 'white',
 21: 'polka dot',
 22: 'man and woman',
 23: 'a man',
 24: 'he is the speaker',
 25: 'at the podium',
 26: 'white',
 27: 'shopping',
 28: 'black',
 29: 'clothes',
 30: 'boy',
 31: 'looking at books',
 32: 'hanging a sign',
 33: 'on the roof',
 34: 'ladders',
 35: 'parking lot',
 36: 'next to the man',
 37: 'crossed legs',
 38: 'left',
 39: 'red',
 40: 'a bag',
 41: 'supermarket',
 42: "person's wrist",
 43: 'bananas',
 44: 'cart',
 45: 'watching the race',
 46: 'rowing',
 47: 'yellow',
 48: 'food',
 49: 'bread',
 50: 'woman in black',
 51: 'donuts',
 52: 'new years eve',
 53: 'a bag',
 54: 'two',
 55: 'park',
 56: 'swingi

In [8]:
with open('./data/train/evjvqa_train_lang.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)
    
annotations = train_data['annotations']

In [26]:
train_answer_dict = {}

for anno in annotations:
    if anno['language'] == 'en':
        train_answer_dict[anno['id']] = anno['answer']

In [16]:
with open('./outputs/results-train.json', 'w', encoding='utf-8') as f:
    json.dump(answer_dict, f, indent=4, ensure_ascii=False)

In [22]:
len(answer_dict), len(train_answer_dict)

(7194, 7194)

In [27]:
from evaluate_metrics import compute_acc

acc = compute_acc(a_gold=train_answer_dict, a_pred=answer_dict)

acc

0.18083837783088338

In [9]:
tmp_annotations = []

for i, anno in tqdm(enumerate(annotations)):
    if anno['language'] == 'en':
        tmp_annotations.append(anno)
        


annotations = tmp_annotations

with open('./data/train/evjvqa_train_lang-en.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, indent=4, ensure_ascii=False)

0it [00:00, ?it/s]